In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe
import openrtdynamics2.targets as tg

from helper_fn import * 

# Loop and yield output samples

In [2]:
dy.clear()

system = dy.enter_system()

# the diagram

yield_event2 = dy.signal_periodic_impulse(period=4, phase=2)


with dy.sub_loop( max_iterations=1000 ) as system:

    cnt = dy.counter()
    system.set_outputs([ cnt ])
  

    # execute the counter above until yield_event. Theb continue the main system for one time-instant
    # and continue inside this subsystem again.
    yield_event = dy.signal_periodic_impulse(period=4, phase=2)
    system.loop_yield( yield_event )


cnt = system.outputs[0]


# define output(s)
dy.append_output(cnt, 'cnt')
dy.append_output(yield_event2, 'yield_event2')


# generate code
code_gen_results = dy.generate_code(template=tg.TargetCppMinimal())

#


compiling system Sys1000 (level 1)... 
compiling system simulation (level 0)... 


In [3]:
compiled_system = dyexe.CompiledCode(code_gen_results)
sim_results = dyexe.run_batch_simulation(dyexe.SystemInstance(compiled_system), input_data={}, N=30)

In [4]:
sim_results['cnt']

array([  2.,   6.,  10.,  14.,  18.,  22.,  26.,  30.,  34.,  38.,  42.,
        46.,  50.,  54.,  58.,  62.,  66.,  70.,  74.,  78.,  82.,  86.,
        90.,  94.,  98., 102., 106., 110., 114., 118.])

In [5]:
assert_equal( sim_results['cnt'] , [ 2.,   6.,  10.,  14.,  18.,  22.,  26.,  30.,  34.,  38.,  42.,
        46.,  50.,  54.,  58.,  62.,  66.,  70.,  74.,  78.,  82.,  86.,
        90.,  94.,  98., 102., 106., 110., 114., 118. ] )

In [6]:
sim_results['yield_event2']

array([0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])

In [7]:
assert_equal( sim_results['yield_event2'] , [ 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0. ] )

# Loop until

In [8]:
dy.clear()

system = dy.enter_system()

# the diagram
count_till = dy.counter()

with dy.sub_loop( max_iterations=1000 ) as loop:

    cnt = dy.counter()    
    
    # compute:
    #
    # accumulated_cnt = 0 + 1 + 2 + 3 + ... + count_till
    accumulated_cnt = dy.sum(cnt, no_delay=True)

    loop.set_outputs([ cnt, accumulated_cnt ])
    loop.loop_until( dy.counter() >= count_till )


cnt             = loop.outputs[0]
accumulated_cnt = loop.outputs[1]

# define output(s)
dy.append_output(cnt, 'cnt')
dy.append_output(accumulated_cnt, 'accumulated_cnt')

# generate code
code_gen_results = dy.generate_code(template=tg.TargetCppMinimal())

#


compiling system Sys1000 (level 1)... 
compiling system simulation (level 0)... 


In [9]:
compiled_system = dyexe.CompiledCode(code_gen_results)
sim_results = dyexe.run_batch_simulation(dyexe.SystemInstance(compiled_system), input_data={}, N=30)

In [10]:
sim_results['cnt']

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29.])

In [11]:
assert_equal( sim_results['cnt'] , [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29 ] )

In [12]:
sim_results['accumulated_cnt']

array([  0.,   1.,   3.,   6.,  10.,  15.,  21.,  28.,  36.,  45.,  55.,
        66.,  78.,  91., 105., 120., 136., 153., 171., 190., 210., 231.,
       253., 276., 300., 325., 351., 378., 406., 435.])

In [13]:
assert_equal( sim_results['accumulated_cnt'] , [ 0.,   1.,   3.,   6.,  10.,  15.,  21.,  28.,  36.,  45.,  55.,
        66.,  78.,  91., 105., 120., 136., 153., 171., 190., 210., 231.,
       253., 276., 300., 325., 351., 378., 406., 435. ] )